# След космического луча | Хакатон Nuclear IT Hack МИФИ

## Пайплайн

In [57]:
import catboost
import pandas as pd

from sklearn.model_selection import train_test_split

import numpy as np

from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
def read_binarry(file_path):
    import struct
    import math

    # Информация о первичной частице
    PART0 = []
    E0 = []

    # Целевые переменные
    angle_tetta = []
    angle_phi = []
    coordinate_x = []
    coordinate_y = []
    power = []
    age = []

    # Количество различных вторичных частиц
    NGAM = []
    NEL = []
    NHADR = []
    NMU = []

    # Нижний/верхний. Энерговыделение в счетчиках СКТ
    EdepCntSCT = []

    # Энерговыделение на 
    energy = []

    with open(file_path, 'rb') as binary_file:
        for _ in range(100000):
            binary_file.read(4 * 3)

            p = struct.unpack('f', binary_file.read(4))[0]
            PART0.append(p)

            E = struct.unpack('f', binary_file.read(4))[0]
            E0.append(E)

            tetta = struct.unpack('f', binary_file.read(4))[0]
            angle_tetta.append(tetta)

            phi = struct.unpack('f', binary_file.read(4))[0]
            angle_phi.append(phi)

            x0 = struct.unpack('f', binary_file.read(4))[0]
            coordinate_x.append(x0)

            y0 = struct.unpack('f', binary_file.read(4))[0]
            coordinate_y.append(y0)

            binary_file.read(4 * 1)

            NGAM.append(struct.unpack('f', binary_file.read(4))[0])
            NEL.append(struct.unpack('f', binary_file.read(4))[0])
            NHADR.append(struct.unpack('f', binary_file.read(4))[0])
            NMU.append(struct.unpack('f', binary_file.read(4))[0])

            power_eas = struct.unpack('f', binary_file.read(4))[0]
            power.append(math.log10(power_eas))

            age_eas = struct.unpack('f', binary_file.read(4))[0]
            age.append(age_eas)

            binary_file.read(4 * 845)

            EdepCntSCT.append(struct.unpack('f' * 2, binary_file.read(4 * 2)))

            binary_file.read(4 * 718)

            energy_release = struct.unpack('f' * 36, binary_file.read(4 * 36))
            energy.append(energy_release)

            binary_file.read(4)
            t = struct.unpack('f' * 144, binary_file.read(4 * 144))
            threshold_time = t[::4]

    df = pd.DataFrame({
        'part0': PART0,
        'E0': E0,
        'power': power,
        'age': age,
        'x': coordinate_x,
        'y': coordinate_y,
        'NGAM' : NGAM,
        'NEL' : NEL,
        'NHADR' : NHADR,
        'NMU' : NMU,
        'tetta': angle_tetta,
        'edep': EdepCntSCT,
        'phi': angle_phi,
        'energy': energy,
    })

    return df

In [ ]:
data = read_binarry('spe27p_100k_2022_correct.dat')

# Рассредоточение кортежа информации на отельные факторы
data[[f'x{i+1}' for i in range(36)]] = pd.DataFrame(data['energy'].tolist(), index=data.index) # для энергии
data[[f'y{i+1}' for i in range(2)]] = pd.DataFrame(data['edep'].tolist(), index=data.index)    # для энерговыделение в счетчиках СКТ

# Синус азимута
data['phi_sin'] = np.sin(np.deg2rad(data['phi']))

# Сумма всех вторичных частиц
data['secondary_particles_amount'] = data['NEL'] + data['NGAM'] + data['NMU']

# Данные о кортеже энергии
data['max_energy'] =  data['energy'].apply(max)
data['sum_energy'] =  data['energy'].apply(sum)
data['mean_energy'] = data['energy'].apply(lambda x: np.mean(x))

data.drop(columns=['phi'], inplace=True)

energy = data['energy'].values.tolist()
data.drop(columns=['energy'], inplace=True)
data.drop(columns=['edep'], inplace=True)

In [180]:
targets = ['power', 'age', 'tetta', 'phi_sin', 'x', 'y']
ys = []

for y in targets:
    ys.append(data[y])

X = data.drop(columns=targets)

In [181]:
splits = []

for y in ys:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    splits.append([X_train, X_test, y_train, y_test])

In [182]:
model = catboost.CatBoostRegressor(
    iterations=20_000,
    learning_rate=0.05,
    early_stopping_rounds=1000,
    depth=3,
    task_type="GPU",
    verbose=5_000)

In [184]:
for i in range(len(targets)):
    print(targets[i])
    model.fit(splits[i][0], splits[i][2])
    print(mean_absolute_error(model.predict(splits[i][1]), splits[i][3]))
    print(r2_score(splits[i][3], model.predict(splits[i][1])))
    print()

power
0:	learn: 0.5837168	total: 1.96ms	remaining: 39.2s
5000:	learn: 0.0389605	total: 7.43s	remaining: 22.3s
10000:	learn: 0.0368213	total: 14.7s	remaining: 14.7s
15000:	learn: 0.0352534	total: 21.8s	remaining: 7.28s
19999:	learn: 0.0339486	total: 28.8s	remaining: 0us
0.029240732728018578
0.9949173184182991

age
0:	learn: 0.0633554	total: 1.62ms	remaining: 32.4s
5000:	learn: 0.0259219	total: 7.29s	remaining: 21.9s
10000:	learn: 0.0239153	total: 14.5s	remaining: 14.5s
15000:	learn: 0.0227824	total: 21.6s	remaining: 7.2s
19999:	learn: 0.0218908	total: 28.7s	remaining: 0us
0.01990800853824355
0.8310824078709992

tetta
0:	learn: 11.5927763	total: 1.66ms	remaining: 33.2s
5000:	learn: 5.7673933	total: 7.62s	remaining: 22.9s
10000:	learn: 5.3907612	total: 15.7s	remaining: 15.7s
15000:	learn: 5.1242926	total: 23.6s	remaining: 7.86s
19999:	learn: 4.9073093	total: 31.3s	remaining: 0us
4.45401145108177
0.7459592531829253

phi_sin
0:	learn: 0.7060958	total: 2.1ms	remaining: 42.1s
5000:	learn: 0.5

___

# Определение X,Y по средневзвешенному

In [29]:
cluster_centers = np.array([
    [-30.0, 0.0],
    [-30.0, 30.0],
    [0.0, 0.0],
    [30, 5],
    [30, 38],
    [3, -65],
    [-15, 50],
    [35, -25],
    [-30, -30]])

In [30]:
def sum_energy_by_cluster(energy_tuple):
    energies = np.array(energy_tuple)
    cluster_energies = energies.reshape(9, 4)
    cluster_sums = np.sum(cluster_energies, axis=1)
    
    return cluster_sums

In [37]:
weights = [sum_energy_by_cluster(x) for x in energy]
res_x = []
res_y = []

for idx in range(len(weights)):
    weighted_sum_x = np.sum(cluster_centers[:, 0] * weights[idx])
    weighted_sum_y = np.sum(cluster_centers[:, 1] * weights[idx])
    total_weight = np.sum(weights[idx])

    result_x = weighted_sum_x / total_weight
    result_y = weighted_sum_y / total_weight

    res_x.append(result_x)
    res_y.append(result_y)


C:\Users\Artem Fedulov\AppData\Local\Temp\ipykernel_14700\226968702.py:10: RuntimeWarning: invalid value encountered in scalar divide
  result_x = weighted_sum_x / total_weight
C:\Users\Artem Fedulov\AppData\Local\Temp\ipykernel_14700\226968702.py:11: RuntimeWarning: invalid value encountered in scalar divide
  result_y = weighted_sum_y / total_weight


In [ ]:
valid_mask = ~np.isnan(res_x) & ~np.isnan(res_y)

mae_x = mean_absolute_error(
    np.array(data['x'])[valid_mask], 
    np.array(res_x)[valid_mask]
)
mae_y = mean_absolute_error(
    np.array(data['y'])[valid_mask],
    np.array(res_y)[valid_mask]
)

print(f"MAE X: {mae_x:.2f}, MAE Y: {mae_y:.2f}")

MAE X: 9.92, MAE Y: 15.88


___

# Сохранение обученных моделей

In [187]:
import pickle

for y in ys:
    model.fit(X, y)
    model_pkl_file = f"model/{y.name}.pkl"
    with open(model_pkl_file, 'wb') as file: pickle.dump(model, file) 

0:	learn: 0.5835065	total: 1.58ms	remaining: 31.7s
5000:	learn: 0.0394028	total: 7.53s	remaining: 22.6s
10000:	learn: 0.0375271	total: 14.8s	remaining: 14.8s
15000:	learn: 0.0361659	total: 22.2s	remaining: 7.41s
19999:	learn: 0.0350430	total: 29.2s	remaining: 0us
0:	learn: 0.0633207	total: 2.72ms	remaining: 54.3s
5000:	learn: 0.0260607	total: 7.08s	remaining: 21.2s
10000:	learn: 0.0242205	total: 14.2s	remaining: 14.2s
15000:	learn: 0.0232303	total: 21.4s	remaining: 7.13s
19999:	learn: 0.0224629	total: 28.3s	remaining: 0us
0:	learn: 11.5769867	total: 3.15ms	remaining: 1m 2s
5000:	learn: 5.8174419	total: 7.13s	remaining: 21.4s
10000:	learn: 5.4805725	total: 14.3s	remaining: 14.3s
15000:	learn: 5.2456634	total: 21.4s	remaining: 7.14s
19999:	learn: 5.0559087	total: 28.2s	remaining: 0us
0:	learn: 0.7064281	total: 1.68ms	remaining: 33.6s
5000:	learn: 0.5995473	total: 7.16s	remaining: 21.5s
10000:	learn: 0.5733538	total: 15.2s	remaining: 15.2s
15000:	learn: 0.5537263	total: 22.9s	remaining: 7